In [ ]:
import requests
import pandas as pd
import numpy as np
import bt

API_KEY_FMP = 'YOUR_FINANCIAL_MODELING_PREP_API_KEY'
tickers = [
    'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AMD', 'AMGN', 'AMZN', 'ASML', 'AVGO',
    'BA', 'BAC', 'BLK', 'BRK.B', 'C', 'CAT', 'CMCSA', 'COST', 'CRM', 'CSCO',
    'CVX', 'DIS', 'F', 'GE', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM',
    'INTC', 'INTU', 'JNJ', 'JPM', 'KO', 'LLY', 'LMT', 'MA', 'MCD', 'MDT',
    'META', 'MKL', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL',
    'PEP', 'PFE', 'PG', 'PM', 'QCOM', 'SBUX', 'TSLA'
]

In [ ]:
def get_price_history_fmp(ticker, api_key=API_KEY_FMP):
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?apikey={api_key}&serietype=line"
    r = requests.get(url)
    data = r.json()
    
    if 'historical' not in data:
        print(f"[!] No se pudo obtener datos para {ticker}")
        return None

    df = pd.DataFrame(data['historical'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.sort_index()

    # Eliminar fechas duplicadas (nos quedamos con la última si hay repetidas)
    df = df[~df.index.duplicated(keep='last')]
    
    return df['close']

In [ ]:
precios = pd.DataFrame()

for t in tickers:
    try:
        serie = get_price_history_fmp(t)
        if serie is not None:
            precios[t] = serie
    except Exception as e:
        print(f"[!] Error con {t}: {e}")

In [ ]:
def get_fundamentals_fmp(ticker, api_key=API_KEY_FMP):
    fundamentals = {}

    # Perfil básico
    url_profile = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}"
    r1 = requests.get(url_profile).json()

    if isinstance(r1, list) and len(r1) > 0:
        d = r1[0]
        fundamentals['ticker'] = ticker
        fundamentals['pe_ratio'] = pd.to_numeric(d.get('pe'), errors='coerce')
        fundamentals['pb_ratio'] = pd.to_numeric(d.get('priceToBookRatio'), errors='coerce')
        fundamentals['market_cap'] = d.get('mktCap')
    else:
        print(f"[!] No se pudo obtener perfil para {ticker}")
        return None

    # ROE desde ratios
    url_ratios = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{ticker}?apikey={api_key}"
    r2 = requests.get(url_ratios).json()

    if isinstance(r2, list) and len(r2) > 0:
        fundamentals['roe'] = r2[0].get('returnOnEquityTTM')
    else:
        print(f"[!] No se pudo obtener ROE para {ticker}")
        fundamentals['roe'] = np.nan

    return fundamentals

# Obtener fundamentos para todos los tickers
fundamentals = []

for t in tickers:
    info = get_fundamentals_fmp(t)
    if info:
        fundamentals.append(info)

df_fundamentals = pd.DataFrame(fundamentals).set_index('ticker')

In [ ]:
def calcular_score(df):
    df = df.copy()

    # Asegurar que los valores son numéricos
    for col in ['market_cap', 'momentum', 'roe', 'volatility']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Ranking
    df['rank_size'] = df['market_cap'].rank(ascending=True)
    df['rank_volatility'] = df['volatility'].rank(ascending=True)
    df['rank_momentum'] = df['momentum'].rank(ascending=False)
    df['rank_roe'] = df['roe'].rank(ascending=False)

    # Score total sin pe_ratio
    df['score_total'] = (
        df['rank_size'] +
        df['rank_volatility'] +
        df['rank_momentum'] +
        df['rank_roe']
    )

    return df

In [ ]:
def calcular_metricas_por_ticker(ticker):
    try:
        # Obtener datos de precios
        precios = get_price_history_fmp(ticker)
        if precios is None:
            return None
        
        # Calcular métricas
        momentum_12m = precios.pct_change(252, fill_method=None).iloc[-1]
        volatilidad_3m = precios.pct_change(fill_method=None).rolling(63).std().iloc[-1]
        
        # Obtener fundamentos
        fundamentos = get_fundamentals_fmp(ticker)
        if fundamentos is None:
            return None
        
        return {
            'ticker': ticker,
            'momentum': momentum_12m,
            'volatility': volatilidad_3m,
            'roe': fundamentos.get('roe', np.nan),
            'market_cap': fundamentos.get('market_cap', np.nan)
        }
    except Exception as e:
        print(f"Error calculando métricas para {ticker}: {e}")
        return None

In [ ]:
# Calcular métricas para todos los tickers
metricas = []
for ticker in tickers:
    resultado = calcular_metricas_por_ticker(ticker)
    if resultado:
        metricas.append(resultado)

df_metricas = pd.DataFrame(metricas)
df_metricas = df_metricas.dropna()

# Aplicar scoring
df_scored = calcular_score(df_metricas)

# Top N tickers
top_n = 10
top_n_tickers = df_scored.nsmallest(top_n, 'score_total')['ticker'].tolist()
print(top_n_tickers)

In [ ]:
# Backtesting
precios_backtest = precios[top_n_tickers].copy()

# Estrategia: Equally weighted portfolio
estrategia = bt.Strategy('Factor Top N',
    [
        bt.algos.RunOnce(),
        bt.algos.SelectAll(),
        bt.algos.WeighEqually(),
        bt.algos.Rebalance()
    ]
)

bt_test = bt.Backtest(estrategia, precios_backtest)

In [ ]:
# Benchmark: SPY
spy = get_price_history_fmp('SPY')
precios_backtest['SPY'] = spy
precios_backtest = precios_backtest.dropna()

In [ ]:
estrategia_spy = bt.Strategy('Benchmark SPY',
    [
        bt.algos.RunOnce(),
        bt.algos.SelectAll(),
        bt.algos.WeighEqually(),
        bt.algos.Rebalance()
    ]
)

bt_spy = bt.Backtest(estrategia_spy, precios_backtest[['SPY']])

In [ ]:
res = bt.run(bt_test, bt_spy)

# Mostrar métricas comparativas
res.display()

# Graficar comparación
res.plot(title='Estrategia Factor Top N vs SPY')